In [ ]:
import torch

from datetime import datetime
from pprint import pprint

from config import *
from data.extract import DetectorDataProvider, LookUpTable

%reload_ext autoreload
%autoreload 2


In [ ]:
START_DATE = datetime.strptime(CONFIG['START_DATE'], DATE_FORMAT)
END_DATE = datetime.strptime(CONFIG['END_DATE'], DATE_FORMAT)

### Read data from the data set

Check list of all sections

In [ ]:
lut = LookUpTable('Data')
sections = lut.get_sections()
pprint(list(sections)[:10])
print(len(sections))

Extract list of detectors for each section

In [ ]:
int_det = lut.get_detectors_per_section()
int_det

Extract counts for all sections

In [ ]:
ddp = DetectorDataProvider('Data')

int_det['Counts'] = int_det.apply(lambda sec: ddp.get_counts_entering_section(sec['End'], sec['Detectors'], START_DATE, END_DATE), axis=1)

int_det.to_hdf(out_path('int_det.hdf'), 'int_det')
int_det

### Write data into a matrix

Write counts into a matrix
$$Q=[q_1,q_2,...,q_p]$$
for
$$q_i=\set{z(s_i,t_1),z(s_i,t_2),...z(s_i,t_d)}^T,$$
$z(s_i,t_j)$ is the traffic flow of the road section $s_i$ within the time interval $(t_0,t_j)$

In [ ]:
mat_q = torch.tensor(int_det['Counts'].tolist()).T
torch.save(mat_q, out_path('mat_q.pt'))
mat_q.shape